In [4]:
#Yadukrishnan - 21BAI1210
from tensorflow import keras
import tensorflow as tf
import os,datetime
import tensorflow_datasets as tfds

In [5]:
#Loading the data from tensorflow_datasets
df, info = tfds.load('patch_camelyon', with_info = True, as_supervised = True)

#Getting the train, validation and test data
train_data = df['train']
valid_data = df['validation']
test_data = df['test']

2024-01-19 04:29:59.415048: W tensorflow/tsl/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata.google.internal".


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

...

Generating train examples...:   0%|          | 0/262144 [00:00<?, ? examples/s]

Shuffling /home/u782d71a9465ceb2764971ba7db30605/tensorflow_datasets/patch_camelyon/2.0.0.incomplete2RENP7/pat…

Generating validation examples...:   0%|          | 0/32768 [00:00<?, ? examples/s]

Shuffling /home/u782d71a9465ceb2764971ba7db30605/tensorflow_datasets/patch_camelyon/2.0.0.incomplete2RENP7/pat…

Dataset patch_camelyon downloaded and prepared to /home/u782d71a9465ceb2764971ba7db30605/tensorflow_datasets/patch_camelyon/2.0.0. Subsequent calls will reuse this data.


In [6]:
#A function to help scale the images
def preprocess(image, labels):
  image = tf.cast(image, tf.float32)
  image /= 255.
  return image, labels

#Applying the preprocess function we the use of map() method
train_data = train_data.map(preprocess)
valid_data = valid_data.map(preprocess)
test_data = test_data.map(preprocess)

#Shuffling the train_data
buffer_size = 1000
train_data = train_data.shuffle(buffer_size)

#Batching and prefetching
batch_size = 128
train_data = train_data.batch(batch_size).prefetch(1)
valid_data = valid_data.batch(batch_size).prefetch(1)
test_data = test_data.batch(batch_size).prefetch(1)

In [7]:
#Seperating image and label into different variables
train_images, train_labels = next(iter(train_data))
valid_images, valid_labels = next(iter(valid_data))
test_images, test_labels  = next(iter(test_data))

#Checking the label shape
valid_labels.shape

#Checking the image shape
train_images.shape

TensorShape([128, 96, 96, 3])

In [8]:
#import the necessary libraries
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization, Dropout
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers

model = Sequential([
                    Conv2D(256, 3,padding='same', kernel_initializer='he_uniform', activation='relu', input_shape = [96, 96, 3]),
                    MaxPooling2D(2),
                    Conv2D(256, 3,padding='same', kernel_initializer='he_uniform',activation='relu',),
                    MaxPooling2D(2),
                    Conv2D(512, 3,padding='same',kernel_initializer='he_uniform',activation='relu',),
                    MaxPooling2D(2),
                    Conv2D(512, 3,padding='same',kernel_initializer='he_uniform',activation='relu',),
                    MaxPooling2D(2),
                    Conv2D(1024, 3,padding='same', kernel_initializer='he_uniform',activation='relu',),
                    MaxPooling2D(2),
                    Conv2D(1024, 3,padding='same', kernel_initializer='he_uniform',activation='relu',),
                    MaxPooling2D(2),
                    Flatten(),
                    Dense(1028,kernel_initializer='he_uniform',activation = 'relu'),
                    Dense(512,kernel_initializer='he_uniform',activation = 'relu'),
                    Dense(128, kernel_initializer='he_uniform',activation = 'relu'),
                    Dense(1, activation = 'sigmoid'),
                    ])

In [9]:
#Compiling our model
model.compile(optimizer= optimizers.Adam(1e-4), loss='binary_crossentropy', metrics=['acc'])

#Callbacks
early_stopping_cb = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

#Fitting our model
history = model.fit( train_images,
                    train_labels,
                    epochs = 30,
                    callbacks=[early_stopping_cb],
                    validation_data = (valid_images, valid_labels),
                    verbose=2)
#21BAI1210

Epoch 1/30


2024-01-19 04:45:02.030357: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type XPU is enabled.


4/4 - 5s - loss: 1.4346 - acc: 0.5781 - val_loss: 0.6765 - val_acc: 0.6484 - 5s/epoch - 1s/step
Epoch 2/30


2024-01-19 04:45:06.584018: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type XPU is enabled.


4/4 - 0s - loss: 1.1134 - acc: 0.4922 - val_loss: 0.6820 - val_acc: 0.5703 - 253ms/epoch - 63ms/step
Epoch 3/30
4/4 - 0s - loss: 0.6884 - acc: 0.4922 - val_loss: 0.9426 - val_acc: 0.4141 - 254ms/epoch - 64ms/step
Epoch 4/30
4/4 - 0s - loss: 0.7690 - acc: 0.4219 - val_loss: 0.6854 - val_acc: 0.5312 - 253ms/epoch - 63ms/step
Epoch 5/30
4/4 - 0s - loss: 0.7037 - acc: 0.5234 - val_loss: 0.6234 - val_acc: 0.5938 - 257ms/epoch - 64ms/step
Epoch 6/30
4/4 - 0s - loss: 0.6359 - acc: 0.6562 - val_loss: 0.6801 - val_acc: 0.5859 - 248ms/epoch - 62ms/step
Epoch 7/30
4/4 - 0s - loss: 0.5718 - acc: 0.7109 - val_loss: 0.5712 - val_acc: 0.7109 - 255ms/epoch - 64ms/step
Epoch 8/30
4/4 - 0s - loss: 0.5779 - acc: 0.6562 - val_loss: 0.5521 - val_acc: 0.7031 - 253ms/epoch - 63ms/step
Epoch 9/30
4/4 - 0s - loss: 0.5292 - acc: 0.7344 - val_loss: 0.5476 - val_acc: 0.7266 - 247ms/epoch - 62ms/step
Epoch 10/30
4/4 - 0s - loss: 0.4844 - acc: 0.7812 - val_loss: 0.5463 - val_acc: 0.6953 - 250ms/epoch - 63ms/step
Ep